# Parrot Diffusion Studies 🦜🎨

This notebook is designed to study parameters, settings and prompts against the [Stable Diffusion model](https://github.com/CompVis/stable-diffusion).

This notebook maintained by [Stephen Young](https://twitter.com/KyrickYoung) or SteveTheNinja#0616

Visit the Parrot Zone [parrotzone.art](http://www.parrotzone.art)

## Changelog
### 2.0
- paperspace port!

In [ ]:
# Create images out

from pathlib import Path
import os


try:
    from google.colab import drive
    google_drive = True
except:
    google_drive = False

if google_drive:
    ROOT_FOLDER = "generative"
    PROJECT_FOLDER = "parrot_studies"
    MOUNTED_PATH = Path('/content/drive')
    MYDRIVE_PATH = MOUNTED_PATH / "MyDrive"
    PROJECT_PATH = MYDRIVE_PATH /  ROOT_FOLDER / PROJECT_FOLDER

    IMAGES_OUT = PROJECT_PATH/ "images_out"

    drive.mount(str(MOUNTED_PATH))

    os.makedirs(PROJECT_PATH, exist_ok=True)
    os.makedirs(IMAGES_OUT, exist_ok=True)
else:
    # setup paths
    # REPLACE THIS WITH YOUR PATH
    # HINT: type $PWD in a terminal
    NOTEBOOK_PATH = Path("/notebooks")
    IMAGES_OUT = NOTEBOOK_PATH / "images_out"

    os.makedirs(IMAGES_OUT, exist_ok=True)

print("Will write output images to folder", IMAGES_OUT)

In [ ]:
#@title Setup
#  scipy ftfy 
%pip install transformers accelerate safetensors omegaconf diffusers[torch]==0.19.0 invisible-watermark>=0.2.0
%pip install pydantic==1.7.4 pillow
%pip install git+https://github.com/kyrick/parrot-tools.git --no-deps

In [ ]:
from parrot_tools.utils import prepare_prompts_for_study
from parrot_tools.generate import run_prompts, BatchSettings

# THIS IS A DIRTY HACK TO SILENCE THE PROGRESS BAR
# THE TQDM PROGRESS BAR BOTTLENECKS CELL OUTPUT AND SLOWS THE NOTEBOOK
from tqdm.auto import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [ ]:
# Initialize the Model

from diffusers import StableDiffusionXLPipeline
import torch

# Instructions:
# 1. Download the safetensors file from hugging face. 
# 2. Upload it to a folder for example a drive location. 
# 3. Get the absolute path to the file and past it here.
# 4. ???
# 5. Profit.
path_to_model = "/root/dev/parrot-tools/model/sd_xl_base_1.0_with_09_vae.safetensors"
pipe = StableDiffusionXLPipeline.from_single_file(
    path_to_model, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipe.to("cuda")
    

In [ ]:
base_prompts = """
a portrait of a character in a scenic environment
a building in a stunning landscape
"""

# Supported formats:
# Gediminas Pranckevicius - first, last
# Aivazovsky, Ivan        - last, first (supports tabs if pasted from google sheets!)
# de Kooning, Willem      - 2+ words in the last name
# Death Burger            - a nickname of two or more words
# RHADS                   - a single nickname
styles_to_study = """
Pranckevicius, Gediminas
Aivazovsky	Ivan
N/A, RHADS
"""
modifiers_to_study = """
graphic novel
vaporwave
"""

# This text will be added to the end of every prompt
append_to_all_prompts = ""

prompts_to_run = prepare_prompts_for_study(base_prompts, styles_to_study, modifiers_to_study, append_to_all_prompts)

# print out the prompts to sanity check
for p in prompts_to_run:
    print(p.prompt, "-", p.base_filename)

In [6]:
#@title Do The Run
#@markdown ## Run Settings
#@markdown set seed to -1 for random seed
batch_name = "StableXLStudy"
#@markdown batch_size is images per prompt
manual_seed = -1
steps = 20
cfg_scale = 7.0

#@markdown ##Image Settings
image_ext = "jpeg"
width = 1024
height = 1024

#@markdown ##Grid Settings
batch_size = 9
grid_cols = 3
grid_padding = 0
#@markdown color name (`black`, `white`, `pink`, etc.) or hex code in format `#00004c'`
grid_bg_color = "white"
save_individual_images = False
display_grid_images = True

batch_settings = BatchSettings(
    batch_size=batch_size,
    batch_name=batch_name,
    base_path=IMAGES_OUT,
    steps=steps,
    cfg_scale=cfg_scale,
    seed=manual_seed,
    image_ext=image_ext,
    image_w=width,
    image_h=height,
    grid_cols=grid_cols,
    grid_padding=grid_padding,
    grid_bg_color=grid_bg_color,
    save_individual_images=save_individual_images,
    display_grid_images=display_grid_images,
)

run_prompts(pipe, prompts_to_run, batch_settings)


KeyboardInterrupt: 